<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Running-ALE" data-toc-modified-id="Running-ALE-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Running ALE</a></span></li><li><span><a href="#Summarizing-results" data-toc-modified-id="Summarizing-results-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Summarizing results</a></span><ul class="toc-item"><li><span><a href="#Writing-combined-ALE-results" data-toc-modified-id="Writing-combined-ALE-results-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Writing combined ALE results</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* run ALE on simulated training dataset for DeepMAsED 
  * the ground truth results can be used for selecting cutoffs for ALE

# Var

In [49]:
work_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/'

# MAGs for genome analysis
asmbl_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly/'

# bam file dir (mapping of metagenome reads to corresponding MAGs)
map_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/'

# ALE executable
ALE_exe = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/bin/ALE/src/ALE'

# params
conda_env = 'base'
threads = 12

# Init

In [42]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(doParallel)

library(future)
library(future.batchtools)
library(future.apply)
options(future.wait.interval = 2.0)

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


# Load

In [6]:
# adding bam files
fasta_files = list.files(asmbl_dir, '*.fasta$', recursive=TRUE, full.names=TRUE)
fasta_files = data.frame(Assembler = fasta_files %>% dirname %>% basename,
                         Rep = fasta_files %>% dirname %>% dirname %>% basename,
                         Fasta = fasta_files)

fasta_files %>% nrow %>% print
fasta_files %>% head(n=3)

[1] 60


Assembler,Rep,Fasta
megahit,1,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//1/megahit/contigs_filtered.fasta
metaspades,1,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//1/metaspades/contigs_filtered.fasta
megahit,10,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//10/megahit/contigs_filtered.fasta


In [8]:
# adding bam files
bam_files = list.files(map_dir, '*.bam$', recursive=TRUE, full.names=TRUE)
bam_files = data.frame(Assembler = bam_files %>% basename %>% gsub('\\.bam$', '', .),
                       Rep = bam_files %>% dirname %>% basename,
                       Bam = bam_files)

bam_files %>% nrow %>% print
bam_files %>% head(n=3)

[1] 60


Assembler,Rep,Bam
megahit,1,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/1/megahit.bam
metaspades,1,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/1/metaspades.bam
megahit,10,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/10/megahit.bam


In [9]:
# joining
MAGs = fasta_files %>%
    inner_join(bam_files, c('Assembler', 'Rep'))
MAGs %>% nrow %>% print
MAGs %>% head(n=3)

[1] 60


Assembler,Rep,Fasta,Bam
megahit,1,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//1/megahit/contigs_filtered.fasta,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/1/megahit.bam
metaspades,1,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//1/metaspades/contigs_filtered.fasta,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/1/metaspades.bam
megahit,10,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//10/megahit/contigs_filtered.fasta,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/10/megahit.bam


In [15]:
# ALE output file name
MAGs = MAGs %>%
    mutate(ALE = mapply(function(A, R) file.path(work_dir, R, paste0(A, '.txt')),
                        A=Assembler, R=Rep))
                        
MAGs %>% head(n=3)

Assembler,Rep,Fasta,Bam,ALE
megahit,1,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//1/megahit/contigs_filtered.fasta,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/1/megahit.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/1/megahit.txt
metaspades,1,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//1/metaspades/contigs_filtered.fasta,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/1/metaspades.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/1/metaspades.txt
megahit,10,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//10/megahit/contigs_filtered.fasta,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/10/megahit.bam,/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/10/megahit.txt


In [30]:
# making output directories
ret = MAGs$ALE %>% 
    dirname() %>%
    unique() %>%
    sapply(., function(x) dir.create(x, showWarnings=FALSE))

/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/1 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/10 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/11 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/12 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/13 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/14 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/15 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/16 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/17 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/18 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/19 
                                                                        FALSE 
 /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/2 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/20 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/21 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/22 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/23 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/24 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/25 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/26 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/27 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/28 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/29 
                                                                        FALSE 
 /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/3 
                                                                        FALSE 
/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/30 
                                                                        FALSE 
 /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/4 
                                                                        FALSE 
 /ebio/abt3_projects/databases_no-backup/DeepMAsED/

# Running ALE

In [31]:
# cluster resources
resources = list(h_rt = '00:59:00',
                 h_vmem = '8G',
                 threads = '1',
                 conda.env = 'py3_physeq')     # conda env with batchtools installed
plan(batchtools_sge, resources=resources, workers=30)

In [32]:
ALE_params = function(x, exe){
    cmd = '{exe} {bam} {fasta} {outfile}'
    cmd = glue::glue(cmd, exe=exe, bam=x[['Bam']], fasta=x[['Fasta']], outfile=x[['ALE']])
    return(cmd)
}

ALE_cmds = apply(MAGs, 1, ALE_params, exe=ALE_exe) %>%
    as.list

ALE_cmds %>% length %>% print
ALE_cmds[[1]]

[1] 60


[1] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/bin/ALE/src/ALE /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/1/megahit.bam /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//1/megahit/contigs_filtered.fasta /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/1/megahit.txt"

In [33]:
bash_job = function(cmd, conda_env, stdout=TRUE, stderr=FALSE){
    # cmd : string; commandline job (eg., 'ls -thlc')
    # conda_env : string; conda environment name
    cmd = sprintf('. ~/.bashrc; conda activate %s; %s', conda_env, cmd)
    cmd = sprintf('-c "%s"', cmd)
    system2('bash', cmd, stdout=stdout, stderr=stderr)
}

In [34]:
# apply function on cluster
job_ret = future_lapply(ALE_cmds, FUN = function(x) bash_job(x, conda_env=conda_env))
job_ret

[[1]]
 [1] "BAM file: /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/map/1/megahit.bam"                                                   
 [2] "Assembly fasta file: /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/assembly//1/megahit/contigs_filtered.fasta"               
 [3] "ALE Output file: /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/1/megahit.txt"                                            
 [4] "Reading in assembly..."                                                                                                                               
 [5] "Reading in the map and computing statistics..."                                                                                                       
 [6] "Insert length and std not given, will be calculated from input map."                                                                                  
 [7] "Found FR sample avg insert length to be 253.739318 from 316378 mapped reads"                                                                          
 [8] "Found FR sample insert length std to be 9.908582"                                                                                                     
 [9] "There were 330559 total reads, 330559 paired (328316 properly mated), 2105 proper singles, 138 improper reads (128 chimeric). (0 reads were unmapped)"
[10] "Saved library parameters to /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/1/megahit.txt.param"                           
[11] "Computing read placements and depths"                                                                                                                 
[12] "FR orientation with 328316 reads, 328316 mapped, 0 unmapped, 328316 placed, 0 mappedButNotPlaced"                                                     
[13] "RF orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                                    
[14] "FF orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                                    
[15] "NOT_PROPER_FR orientation with 2 reads, 2 mapped, 0 unmapped, 2 placed, 0 mappedButNotPlaced"                                                         
[16] "NOT_PROPER_RF orientation with 8 reads, 8 mapped, 0 unmapped, 8 placed, 0 mappedButNotPlaced"                                                         
[17] "NOT_PROPER_FF orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                         
[18] "UNMAPPED_PAIR orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                         
[19] "CHIMER orientation with 128 reads, 128 mapped, 0 unmapped, 128 placed, 0 mappedButNotPlaced"                                                          
[20] "SINGLE_UNMAPPED_MATE orientation with 2105 reads, 2105 mapped, 0 unmapped, 2105 placed, 0 mappedButNotPlaced"                                         
[21] "UNMAPPED_MATE orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                         
[22] "HALF_VALID_MATE orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                       
[23] "SINGLE_READ orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                           
[24] "UNMAPPED_SINGLE orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                       
[25] "NO_READS orientation with 0 reads, 0 mapped, 0 unmapped, 0 placed, 0 mappedButNotPlaced"                                                              
[26] "Total mapped reads: 330559"                                    

# Summarizing results

In [35]:
ALE_res_files = list.files(work_dir, '*.txt$', recursive=TRUE, full.names=TRUE)
ALE_res_files %>% length %>% print
ALE_res_files %>% head

[1] 60


[1] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/1/megahit.txt"    
[2] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/1/metaspades.txt" 
[3] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/10/megahit.txt"   
[4] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/10/metaspades.txt"
[5] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/11/megahit.txt"   
[6] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE/11/metaspades.txt"

In [46]:
read_ALE_files = function(in_file){
    # input command
    cmd = 'egrep -v "^#" {in_file}'
    cmd = glue::glue(cmd, in_file=in_file)
    # reading in table
    df = fread(cmd, sep=' ', header=FALSE)
    # formatting
    colnames(df) = c('contig', 'position', 'depth', 
                     'ln__depthLike', 'ln__placeLike', 
                     'ln__insertLike', 'ln__kmerLike')
    df$Assembler = in_file %>% basename %>% gsub('\\.txt$', '', .)
    df$Rep = in_file %>% dirname %>% basename
    return(df)
}

In [48]:
doParallel::registerDoParallel(threads)
ALE_res = plyr::llply(as.list(ALE_res_files), read_ALE_files, .parallel=TRUE) %>%
    do.call(rbind, .)

ALE_res %>% nrow %>% print
ALE_res %>% head(n=3)

[1] 849228827


contig,position,depth,ln__depthLike,ln__placeLike,ln__insertLike,ln__kmerLike,Assembler,Rep
0,0,1,-2.601,-0.007,0.32,-0.198,megahit,1
0,1,1,-2.601,-0.007,0.32,-0.420,megahit,1
0,2,1,-2.601,-0.007,0.32,-0.393,megahit,1


## Writing combined ALE results

In [50]:
out_file = file.path(work_dir, 'ALE_results.tsv')
ALE_res %>%
    write.table(file=out_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', out_file, '\n')

File written: /ebio/abt3_projects/databases_no-backup/DeepMAsED/train_runs/n1000_r30/ALE//ALE_results.tsv 


# sessionInfo

In [53]:
sessionInfo()

R version 3.4.1 (2017-06-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.1 LTS

Matrix products: default
BLAS: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRblas.so
LAPACK: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] doParallel_1.0.11       iterators_1.0.10        foreach_1.4.4          
 [4] data.table_1.11.4       bindrcpp_0.2.2          future.apply_1.0.0     
 [7] future.batchtools_0.7.1 fut